In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile
import tensorflow as tf
import numpy as np
from qiskit.circuit import Parameter, ParameterVector
import qiskit
from qiskit import quantum_info
from qiskit.execute_function import execute
from qiskit import BasicAer

seed = 10599

sys.path.append('../')

In [2]:
from part2_rodrigo import VQC

In [223]:
images=np.load('../data/images.npy')
labels=np.load('../data/labels.npy')

In [4]:
images_resized = np.resize(images, (2000,4,4))

In [6]:
max_images = np.max(np.abs(images_resized))
images_resized_normalized = images_resized/max_images

In [166]:
def encoder_v2(image,threshold):
    info_image = np.ndarray.flatten(image)
    n_qubits = len(info_image)
    q_register = QuantumRegister(n_qubits)
    qc = QuantumCircuit(q_register, name='Encoder')
    for i, pixel_value in enumerate(info_image):
        if pixel_value>=threshold:
            qc.x( q_register[i])
    return qc

In [243]:
n_layers = 1
n_qubits = 16
p = ParameterVector('$\theta$',3*n_layers*n_qubits)
p = np.random.random(n_layers*n_qubits*3)
my_vqc = VQC(n_layers,n_qubits,p)
vqc_circuit = my_vqc.vqc()

In [244]:
def execute_circuit(qc, shots=1024, device=None):
    if device is None:
        device = Aer.get_backend('qasm_simulator')
    else:
        device = device
    
    circ_trans = transpile(qc,device)
    counts = device.run(circ_trans, shots=shots).result().get_counts()
    
    return counts

In [16]:
def simulate(circuit: qiskit.QuantumCircuit) -> dict:
    """Simulate the circuit, give the state vector as the result."""
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    result = job.result()
    state_vector = result.get_statevector()
    
    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population
    
    return histogram

In [191]:
def histogram_to_category(histogram):
    """This function take a histogram representations of circuit execution results, and process into labels as described in 
    the problem description."""
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
    return positive

In [192]:
from part1 import encoder

In [217]:
def run_part2(image):

    #load the quantum classifier circuit
    classifier=vqc_circuit
    
    #encode image into circuit
    circuit=encoder(image)
    
    #append with classifier circuit
    qc = QuantumCircuit(16,16)
    qc.compose(circuit)
    qc.compose(classifier)
    
    #simulate circuit
    histogram=simulate(qc)
        
    #convert histogram to category
    label=histogram_to_category(histogram)
        
    return circuit,label

In [233]:
from sklearn.metrics import mean_squared_error

In [250]:
circuit=encoder(images[0])
    
#append with classifier circuit
qc = QuantumCircuit(16,16)
qc = qc.compose(circuit)
qc = qc.compose(vqc_circuit)

#simulate circuit
histogram=simulate(qc)
histogram

{18: 4.87925356473887e-09,
 32: 7.513335974802891e-09,
 33: 3.3107098784536987e-09,
 34: 9.869228197144309e-08,
 35: 2.3865431429612886e-09,
 36: 6.721584305142568e-09,
 37: 2.961828890991214e-09,
 38: 9.324455593240369e-09,
 40: 1.8772938481318084e-08,
 41: 8.272191352301925e-09,
 42: 2.4659407542059347e-07,
 43: 5.963053929183456e-09,
 44: 1.9869729867917684e-07,
 45: 8.755486401318207e-08,
 46: 2.7564098788633774e-07,
 47: 6.665456471556305e-09,
 48: 2.5905590083890595e-08,
 49: 1.141515477086845e-08,
 50: 3.402858343843763e-07,
 51: 8.228676128209075e-09,
 52: 2.3175671673314748e-08,
 53: 1.0212231345165635e-08,
 54: 3.2150235948987106e-08,
 58: 1.1511278406222588e-08,
 60: 9.275405014330507e-09,
 61: 4.087155840040628e-09,
 62: 1.2867219726645886e-08,
 64: 1.0832158293609874e-09,
 66: 1.422870512189891e-08,
 70: 1.3443293275643185e-09,
 72: 2.7065399703647996e-09,
 73: 1.1926218455246974e-09,
 74: 3.5552064597940695e-08,
 76: 2.8646670387477117e-08,
 77: 1.2622996622897015e-08,
 7

In [245]:
def loss(image,label,parameters):
    circuit=encoder(image)
    
    #append with classifier circuit
    qc = QuantumCircuit(16,16)
    qc = qc.compose(circuit)
    qc = qc.compose(vqc_circuit)
    
    #simulate circuit
    histogram=simulate(qc)
        
    #convert histogram to category
    predict = histogram_to_category(histogram)
    #print("Prediction: ", predict)
    #print("Label: ", label)
    return (label-predict)**2

In [246]:
def cost_function(images,labels,parameters):
    cost = []
    labels = [1.0 if labels[i] == True else 0.0 for i in range(len(labels))]
    for i in range(len(images)):
        cost.append(loss(images[i],labels[i],parameters))
    return np.mean(cost)

In [247]:
from sklearn.utils import shuffle
images, labels = shuffle(images, labels)

In [248]:
p = np.random.random(3*n_qubits*n_layers)
dataset = images[0:10]
correct_labels = labels[0:10]
c = cost_function(dataset,correct_labels,p)
print("cost for theta={} ----> {}".format(p,c))

AssertionError: 

In [231]:
from qiskit.algorithms.optimizers import COBYLA, ADAM, SPSA #...

# Callback function for optimiser for plotting purposes
def store_intermediate_result(evaluation, parameter, cost, 
                              stepsize, accept):
    evaluations.append(evaluation)
    parameters.append(parameter)
    costs.append(cost)

parameters = []
costs = []
evaluations = []

optimizer = SPSA(maxiter=50,callback=store_intermediate_result)

p = np.random.random(3*n_qubits*n_layers)

objective_function = lambda p: cost_function(images[0:50],labels[0:50],p)
                                              
ret = optimizer.optimize(num_vars=3*n_qubits*n_layers, objective_function=objective_function, initial_point=p)

print("OPTIMIZATION COMPLETED! RESULT ---> {}".format(ret))

C:\Users\rodri\AppData\Local\Temp\ipykernel_24152\3066818006.py:20: DeprecationWarning:

The SPSA.optimize method is deprecated as of Qiskit Terra 0.21.0 and will be removed no sooner than 3 months after the release date. Instead, use SPSA.minimize as a replacement, which supports the same arguments but follows the interface of scipy.optimize and returns a complete result object containing additional information.

c:\Users\rodri\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit\algorithms\optimizers\spsa.py:345: RuntimeWarning:

divide by zero encountered in double_scalars

c:\Users\rodri\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit\algorithms\optimizers\spsa.py:565: RuntimeWarning:

invalid value encountered in multiply



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\rodri\AppData\Local\Temp\ipykernel_24152\3066818006.py", line 20, in <cell line: 20>
    ret = optimizer.optimize(num_vars=3*n_qubits*n_layers, objective_function=objective_function, initial_point=p)
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit\utils\deprecation.py", line 49, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit\algorithms\optimizers\spsa.py", line 679, in optimize
    result = self.minimize(objective_function, initial_point)
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit\algorithms\optimizers\spsa.py", line 556, in minimize
    fx_estimate, update = self._compute_updat